<a href="https://colab.research.google.com/github/IvanKatorgin/Deep-Learning/blob/main/Deep_Learning_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Курс "Deep Learning". Домашнее задание 7. Рекуррентные сети 2. Каторгин И.П.

## Задание

Задание 1

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:

x - последовательность цифр

y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
- научить модель предсказывать y(i) по x(i)
- пробовать RNN, LSTM, GRU

Задание 2 (дополнительное и необязательное)

применить LSTM для решения лекционного практического задания https://colab.research.google.com/drive/1_rNrPHl6sYHNp-xo2G6I_SmRGTSSeICL

In [3]:
# загрузим библиотеки
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, time, random, os, torch, re, tqdm
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch.nn import functional as F
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Задание 1

In [26]:
# функция генерации последовательностей
def generate_sequence(length):
    """Генерирует последовательность x и вычисляет y по правилам задачи"""
    x = [random.randint(0, 9) for _ in range(length)]
    y = [x[0]]  # y1 = x1
    for i in range(1, len(x)):
        yi = x[i] + x[0]
        if yi >= 10:
            yi -= 10
        y.append(yi)
    return x, y

In [27]:
# Пример использования:
x, y = generate_sequence(15)
print("x:", x)
print("y:", y)

x: [0, 0, 1, 9, 0, 1, 9, 2, 7, 6, 9, 2, 2, 6, 4]
y: [0, 0, 1, 9, 0, 1, 9, 2, 7, 6, 9, 2, 2, 6, 4]


Cоздадим обычную полносвязную сеть

In [17]:
# Генерация данных
def generate_data(num_samples=10000, seq_length=10):
    X, Y = [], []
    for _ in range(num_samples):
        x = [random.randint(0, 9) for _ in range(seq_length)]
        y = [x[0]]  # y1 = x1
        for i in range(1, len(x)):
            yi = x[i] + x[0]
            if yi >= 10: yi -= 10
            y.append(yi)
        X.append(x)
        Y.append(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.long)

In [18]:
# Параметры
SEQ_LENGTH = 10
NUM_SAMPLES = 10000
BATCH_SIZE = 64

In [19]:
# Датасет
X, Y = generate_data(NUM_SAMPLES, SEQ_LENGTH)
dataset = TensorDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
# Модель
class DigitPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 128)  # Принимает x(i)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)  # 10 классов (0-9)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

In [22]:
model = DigitPredictor()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
# Обучение
for epoch in range(10):
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        # Обрабатываем каждый элемент последовательности
        loss = 0
        for i in range(SEQ_LENGTH):
            x_i = batch_x[:, i].unsqueeze(1)  # Текущий x(i)
            y_i = batch_y[:, i]  # Текущий y(i)
            outputs = model(x_i)
            loss += criterion(outputs, y_i)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item() / SEQ_LENGTH}")

Epoch 0, Loss: 2.305716705322266
Epoch 1, Loss: 2.2970661163330077
Epoch 2, Loss: 2.295778274536133
Epoch 3, Loss: 2.2559749603271486
Epoch 4, Loss: 2.315299415588379
Epoch 5, Loss: 2.292628860473633
Epoch 6, Loss: 2.248338317871094
Epoch 7, Loss: 2.28228816986084
Epoch 8, Loss: 2.231733703613281
Epoch 9, Loss: 2.2355888366699217


In [24]:
# Проверка
test_x, test_y = generate_data(1, SEQ_LENGTH)
model.eval()
with torch.no_grad():
    preds = []
    for i in range(SEQ_LENGTH):
        output = model(test_x[0, i].unsqueeze(0))
        pred = output.argmax().item()
        preds.append(pred)
    print("Real x:", test_x[0].int().tolist())
    print("Real y:", test_y[0].tolist())
    print("Pred y:", preds)

Real x: [9, 0, 0, 3, 4, 9, 4, 4, 2, 9]
Real y: [9, 9, 9, 2, 3, 8, 3, 3, 1, 8]
Pred y: [9, 0, 0, 3, 4, 9, 4, 4, 2, 9]


Ошибка высокая, предсказанный у практически не совпал с истинным

Создадим RNN модель

In [55]:
# Размер обучающей выборки и длина последовательности
N = 10000
S = 15

# Параметры
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [56]:
# Генерация данных
def generate_sequences(num_samples=N, seq_length=S):
    X, Y = [], []
    for _ in range(num_samples):
        x = [random.randint(0, 9) for _ in range(seq_length)]
        y = [x[0]]  # y1 = x1
        for i in range(1, len(x)):
            yi = x[i] + x[0]
            if yi >= 10: yi -= 10
            y.append(yi)
        X.append(x)
        Y.append(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.long)

In [57]:
# Создание данных
X, Y = generate_sequences()
dataset = TensorDataset(X.unsqueeze(2), Y)  # Добавляем размерность канала
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [58]:
# Вывод обучающей выборки
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("X[0]:", X[0])
print("Y[0]:", Y[0])
print("X[1]:", X[1])
print("Y[1]:", Y[1])
print("X[2]:", X[2])
print("Y[2]:", Y[2])

X.shape: torch.Size([10000, 15])
Y.shape: torch.Size([10000, 15])
X[0]: tensor([5., 9., 2., 4., 4., 3., 1., 3., 7., 2., 6., 0., 5., 7., 0.])
Y[0]: tensor([5, 4, 7, 9, 9, 8, 6, 8, 2, 7, 1, 5, 0, 2, 5])
X[1]: tensor([4., 5., 9., 3., 6., 5., 6., 3., 7., 8., 6., 8., 1., 7., 5.])
Y[1]: tensor([4, 9, 3, 7, 0, 9, 0, 7, 1, 2, 0, 2, 5, 1, 9])
X[2]: tensor([4., 4., 2., 2., 7., 7., 2., 7., 5., 6., 2., 9., 2., 3., 0.])
Y[2]: tensor([4, 8, 6, 6, 1, 1, 6, 1, 9, 0, 6, 3, 6, 7, 4])


In [59]:
# модель RNN
class SequenceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 10)  # 10 классов (цифры 0-9)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)  # [batch, seq_len, hidden_size]
        return self.fc(rnn_out)    # [batch, seq_len, 10]

In [60]:
model = SequenceModel()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [61]:
# Обучение
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x)  # [batch, seq_len, 10]
        loss = criterion(outputs.view(-1, 10), batch_y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1, Loss: 2.3030
Epoch 2, Loss: 2.3015
Epoch 3, Loss: 2.3004
Epoch 4, Loss: 2.2735
Epoch 5, Loss: 2.1780
Epoch 6, Loss: 1.9775
Epoch 7, Loss: 1.6480
Epoch 8, Loss: 1.4062
Epoch 9, Loss: 1.1900
Epoch 10, Loss: 1.0528


In [62]:
# Проверка
def predict(model, x):
    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0).unsqueeze(2)
        outputs = model(x_tensor)
        return outputs.argmax(dim=2).squeeze().tolist()

test_x, test_y = generate_sequences(1, SEQ_LENGTH)
pred_y = predict(model, test_x[0])
print("\nTest:")
print("X:", test_x[0].int().tolist())
print("Real Y:", test_y[0].tolist())
print("Pred Y:", pred_y)


Test:
X: [9, 1, 1, 9, 4, 7, 6, 8, 1, 2]
Real Y: [9, 0, 0, 8, 3, 6, 5, 7, 0, 1]
Pred Y: [8, 9, 9, 6, 3, 7, 4, 7, 0, 1]


По сравнению с обычной полносвязной моделью уже лучше, ошибка значительно меньше, но предсказанный у все равно часто ошибается

Создадим LSTM модель

In [44]:
# Размер обучающей выборки и длина последовательности
N = 10000
S = 15

# Параметры
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [45]:
# Генерация данных
def generate_sequences(num_samples=N, seq_length=S):
    X, Y = [], []
    for _ in range(num_samples):
        x = [random.randint(0, 9) for _ in range(seq_length)]
        y = [x[0]]  # y1 = x1
        for i in range(1, len(x)):
            yi = x[i] + x[0]
            if yi >= 10: yi -= 10
            y.append(yi)
        X.append(x)
        Y.append(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.long)

In [46]:
# Создание данных
X, Y = generate_sequences()
dataset = TensorDataset(X.unsqueeze(2), Y)  # Добавляем размерность канала
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [47]:
# Вывод обучающей выборки
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("X[0]:", X[0])
print("Y[0]:", Y[0])
print("X[1]:", X[1])
print("Y[1]:", Y[1])
print("X[2]:", X[2])
print("Y[2]:", Y[2])

X.shape: torch.Size([10000, 15])
Y.shape: torch.Size([10000, 15])
X[0]: tensor([9., 1., 6., 1., 3., 5., 6., 0., 9., 1., 9., 1., 7., 3., 5.])
Y[0]: tensor([9, 0, 5, 0, 2, 4, 5, 9, 8, 0, 8, 0, 6, 2, 4])
X[1]: tensor([7., 9., 9., 6., 2., 6., 3., 6., 3., 1., 9., 7., 7., 2., 6.])
Y[1]: tensor([7, 6, 6, 3, 9, 3, 0, 3, 0, 8, 6, 4, 4, 9, 3])
X[2]: tensor([3., 2., 1., 9., 0., 2., 6., 5., 2., 4., 1., 2., 2., 7., 0.])
Y[2]: tensor([3, 5, 4, 2, 3, 5, 9, 8, 5, 7, 4, 5, 5, 0, 3])


In [48]:
# Модель LSTM
class SequenceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 10)  # 10 классов (цифры 0-9)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden_size]
        return self.fc(lstm_out)     # [batch, seq_len, 10]

In [49]:
model = SequenceModel()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
# Обучение
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x)  # [batch, seq_len, 10]
        loss = criterion(outputs.view(-1, 10), batch_y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1, Loss: 2.3002
Epoch 2, Loss: 2.2554
Epoch 3, Loss: 2.0442
Epoch 4, Loss: 1.4677
Epoch 5, Loss: 1.0413
Epoch 6, Loss: 0.7861
Epoch 7, Loss: 0.6164
Epoch 8, Loss: 0.4937
Epoch 9, Loss: 0.4008
Epoch 10, Loss: 0.3301


In [54]:
# Проверка
def predict(model, x):
    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0).unsqueeze(2)
        outputs = model(x_tensor)
        return outputs.argmax(dim=2).squeeze().tolist()

test_x, test_y = generate_sequences(1, SEQ_LENGTH)
pred_y = predict(model, test_x[0])
print("\nTest:")
print("X:", test_x[0].int().tolist())
print("Real Y:", test_y[0].tolist())
print("Pred Y:", pred_y)


Test:
X: [3, 4, 4, 9, 9, 4, 0, 4, 4, 0]
Real Y: [3, 7, 7, 2, 2, 7, 3, 7, 7, 3]
Pred Y: [3, 7, 7, 2, 2, 7, 3, 7, 7, 3]


Предсказанный у практически не ошибается и совпадает с истинным

Создадим GRU модель

In [80]:
# Размер обучающей выборки и длина последовательности
N = 10000
S = 15

# Параметры
BATCH_SIZE = 64
NUM_EPOCHS = 10
EMBEDDING_SIZE = 64
HIDDEN_SIZE = 128

In [81]:
# Генерация данных
def generate_sequences(num_samples=N, seq_length=S):
    X, Y = [], []
    for _ in range(num_samples):
        x = [random.randint(0, 9) for _ in range(seq_length)]
        y = [x[0]]  # y1 = x1
        for i in range(1, len(x)):
            yi = x[i] + x[0]
            if yi >= 10: yi -= 10
            y.append(yi)
        X.append(x)
        Y.append(y)
    return torch.tensor(X, dtype=torch.long), torch.tensor(Y, dtype=torch.long)

In [82]:
# Создание данных
X, Y = generate_sequences()
dataset = TensorDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [83]:
# Вывод обучающей выборки
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("X[0]:", X[0])
print("Y[0]:", Y[0])
print("X[1]:", X[1])
print("Y[1]:", Y[1])
print("X[2]:", X[2])
print("Y[2]:", Y[2])

X.shape: torch.Size([10000, 15])
Y.shape: torch.Size([10000, 15])
X[0]: tensor([0, 8, 9, 4, 9, 1, 6, 2, 5, 9, 6, 0, 6, 6, 3])
Y[0]: tensor([0, 8, 9, 4, 9, 1, 6, 2, 5, 9, 6, 0, 6, 6, 3])
X[1]: tensor([7, 8, 9, 9, 2, 4, 4, 9, 2, 5, 5, 1, 5, 6, 0])
Y[1]: tensor([7, 5, 6, 6, 9, 1, 1, 6, 9, 2, 2, 8, 2, 3, 7])
X[2]: tensor([5, 6, 0, 4, 2, 1, 4, 6, 9, 5, 0, 0, 3, 4, 8])
Y[2]: tensor([5, 1, 5, 9, 7, 6, 9, 1, 4, 0, 5, 5, 8, 9, 3])


In [84]:
# Модель GRU с embedding
class GRUModel(nn.Module):
    def __init__(self, vocab_size=10, embed_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        # x shape: [batch, seq_len]
        embedded = self.embedding(x)  # [batch, seq_len, embed_size]
        gru_out, _ = self.gru(embedded)  # [batch, seq_len, hidden_size]
        output = self.fc(gru_out)  # [batch, seq_len, vocab_size]
        return output

In [85]:
model = GRUModel()

In [86]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [87]:
rnn = nn.GRU(10,128, batch_first=True)
o, s = rnn(torch.randn(64, 15, 10))
o.shape, len(s), s[0].shape

(torch.Size([64, 15, 128]), 1, torch.Size([64, 128]))

In [88]:
# Обучение
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x) # [batch, seq_len, 10]
        loss = criterion(outputs.view(-1, 10), batch_y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1, Loss: 2.2070
Epoch 2, Loss: 0.6658
Epoch 3, Loss: 0.0492
Epoch 4, Loss: 0.0179
Epoch 5, Loss: 0.0100
Epoch 6, Loss: 0.0066
Epoch 7, Loss: 0.0047
Epoch 8, Loss: 0.0035
Epoch 9, Loss: 0.0028
Epoch 10, Loss: 0.0022


In [90]:
# Проверка
def predict(model, x):
    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(x, dtype=torch.long).unsqueeze(0)
        outputs = model(x_tensor)
        return outputs.argmax(dim=2).squeeze().tolist()

test_x, test_y = generate_sequences(1, SEQ_LENGTH)
pred_y = predict(model, test_x[0])
print("\nTest:")
print("X:", test_x[0].int().tolist())
print("Real Y:", test_y[0].tolist())
print("Pred Y:", pred_y)


Test:
X: [7, 8, 1, 1, 6, 1, 0, 7, 5, 1]
Real Y: [7, 5, 8, 8, 3, 8, 7, 4, 2, 8]
Pred Y: [7, 5, 8, 8, 3, 8, 7, 4, 2, 8]


Из всех рассмотренных моделей, эта модель наиболее точная

## Задание 2

In [2]:
# загрузим датасет
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [3]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [4]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [5]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [6]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [7]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [8]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

In [9]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)

        # Обратотка как LSTM (состояние кортеж) так и GRU (состоние тензор)
        if isinstance(state, tuple):  # LSTM case
            hidden_state = state[0]  # Берем hidden_state, игнорируем cell_state
        else:  # GRU case
            hidden_state = state

        predictions = self.output(hidden_state.squeeze(0))  # Удаляем пакетный размер при необходимости
        return predictions

In [11]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [12]:
X.shape

torch.Size([193075, 40])

In [13]:
model(X[0:1])

tensor([[-0.0077,  0.0608, -0.0420, -0.0356,  0.0468,  0.0301, -0.0419, -0.0440,
          0.1277,  0.0895, -0.0299, -0.0096,  0.0432, -0.0968, -0.1212,  0.0095,
          0.0048,  0.1074,  0.0044, -0.1479,  0.0375,  0.1041, -0.0058,  0.0702,
         -0.1288, -0.0989,  0.0166]], grad_fn=<AddmmBackward0>)

In [14]:
embedding = nn.Embedding(len(INDEX_TO_CHAR), 15)
rnn = nn.LSTM(15,128, batch_first=True)

In [15]:
o, s = rnn(embedding(X[0:10]))
print(o.shape, s[0].shape, s[1].shape)  # LSTM возвращает кортеж (hidden_state, cell_state)

torch.Size([10, 40, 128]) torch.Size([1, 10, 128]) torch.Size([1, 10, 128])


In [16]:
model = model.cuda()

In [17]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [18]:
generate_text()

f discernment luke improved he that humb|gjrvamgho aqdlyzurjaprvvpcmntqghitmait o


In [19]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [20]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 3.889, Train loss: 2.175
nal influence of this metaphysical neces|tuan our ove ston to ceacing self whe re
Epoch 1. Time: 3.964, Train loss: 1.801
e is far less worthy of respect religion| masted gree a despes foroutd calle loct
Epoch 2. Time: 3.360, Train loss: 1.662
sight there must always exist an inheren|t undimarioubees dur there remandernging
Epoch 3. Time: 3.526, Train loss: 1.576
 ears nowadays are unwilling to hear suc|celed bould for is he dighs peithers in 
Epoch 4. Time: 3.555, Train loss: 1.514
 which rends while it delights what and |nethy that the juch one sholigisio lat a
Epoch 5. Time: 3.811, Train loss: 1.467
on of european taste indeed who could do|rglave our converstionss to exerjectical
Epoch 6. Time: 3.554, Train loss: 1.429
 epistles i x at the risk of displeasing| such obteremost ciluistion does is with
Epoch 7. Time: 3.396, Train loss: 1.398
 also nothing more for men to do that is| appearnce is ording formably is most sp
Epoch 8. Time: 3.973, Tr

Выводы:
1. Построена модель предсказания y(i) по x(i)
2. Реализованы обычная полносвязная сеть, RNN, LSTM, GRU
3. Наиболее точной оказалась модель GRU
4. Применена LSTM для решения лекционного практического задания